## Import Dependencies

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
#spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Get:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-01-06 17:42:53--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.76MB/s    in 0.5s    

2022-01-06 17:42:54 (1.76 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



## Start Session and Import Data

In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Toys").config("spark.driver.extraClassPath","./postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz'
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Toys_v1_00.tsv.gz"), sep="\t", header=True)

# show dataframe
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

## Data Processing

In [5]:
# count number of records (rows) in dataset
print(f"The number of records in the toys reviews dataset is: {df.count()}")

The number of records in the toys reviews dataset is: 4864249


In [6]:
# drop rows with na values
df = df.dropna()

# drop duplicate rows
df = df.dropDuplicates()

# count new number of records
print(f"The number of records in the toys reviews dataset  after cleaning is: {df.count()}")

The number of records in the toys reviews dataset  after cleaning is: 4863497


In [7]:
# pull out fields for review_id_table
review_id_table = df['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date']

# view column dtypes to ensure they match the sql schema
review_id_table.dtypes

[('review_id', 'string'),
 ('customer_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'string'),
 ('review_date', 'string')]

In [8]:
# change dtype of customer_id and product_parent to integer and review_date to date
from pyspark.sql.types import IntegerType
from pyspark.sql.types import DateType
review_id_table = review_id_table.withColumn('customer_id', review_id_table['customer_id'].cast(IntegerType()))
review_id_table = review_id_table.withColumn('product_parent', review_id_table['product_parent'].cast(IntegerType()))
review_id_table = review_id_table.withColumn('review_date', review_id_table['review_date'].cast(DateType()))

# confirm dtypes have been fixed
review_id_table.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

In [9]:
# pull out fields for products table
products = df['product_id', 'product_title']

# drop duplicates to ensure each value only appears once
products = products.dropDuplicates()

# view column dtypes to ensure they match the sql schema
products.dtypes

[('product_id', 'string'), ('product_title', 'string')]

Both columns of this table have the correct datatype

In [10]:
# count number of reviews for each customer
customers = df.groupBy('customer_id').count()

# rename count column to customer_count and change dtypes to int to match sql schema
customers = customers.withColumn('customer_id', customers['customer_id'].cast(IntegerType()))
customers = customers.withColumnRenamed('count', 'customer_count')
customers = customers.withColumn('customer_count', customers['customer_count'].cast(IntegerType()))

# show table
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|     769294|             2|
|   16000435|             6|
|   38872695|             1|
|   30070974|             2|
|    7405342|             4|
|   21957226|            28|
|   29477704|             1|
|   33327462|             1|
|    6573286|             1|
|   16080617|             4|
|   46234257|            11|
|   35148498|             3|
|   10872634|             6|
|   37751531|             1|
|   10845418|             3|
|   13838444|             1|
|   18738011|             4|
|   16148559|             1|
|   51612311|             2|
|   11290755|             2|
+-----------+--------------+
only showing top 20 rows



In [11]:
customers.dtypes

[('customer_id', 'int'), ('customer_count', 'int')]

In [12]:
# pull out fields for vine_table
vine_table = df['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine']

# check column dtypes
vine_table.dtypes

[('review_id', 'string'),
 ('star_rating', 'string'),
 ('helpful_votes', 'string'),
 ('total_votes', 'string'),
 ('vine', 'string')]

In [13]:
# cast the star_rating, helpful_votes, and total_votes columns as integers
vine_table = vine_table.withColumn('star_rating', vine_table['star_rating'].cast(IntegerType()))
vine_table = vine_table.withColumn('helpful_votes', vine_table['helpful_votes'].cast(IntegerType()))
vine_table = vine_table.withColumn('total_votes', vine_table['total_votes'].cast(IntegerType()))

# recheck column dtypes to ensure they have changed to fit the schema
vine_table.dtypes

[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string')]

## Loading dataframes into RDS

In [14]:
# temporarily load in postgres password
sql_password = "" # password removed to preserve security

In [15]:
# import pyspark sql context
from pyspark.sql import SQLContext

# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://database-big-data-challenge-video-games.c2w0aiiywfrx.us-east-2.rds.amazonaws.com:5432/amazon_reviews_toys"
config = {"user":"postgres",
          "password": sql_password,
          "driver":"org.postgresql.Driver"}

In [16]:
# write review_id_table to db table
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [17]:
# write products to db table
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [18]:
# write customers table to db table
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [19]:
# write vine table to db table
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)